In [2]:
import importlib, tools
importlib.reload(tools)
from tools import path_from_root

# %matplotlib qt
%matplotlib inline

In [3]:
from typing import Generator
from PIL import Image
import os
import shutil


def extract_samples_from_file(
    file: str, sample_size: int | tuple[int]
) -> Generator[Image.Image, None, None]:
    if isinstance(sample_size, int):
        sample_size = (sample_size, sample_size)
    with Image.open(file) as img:
        for x in range(int(img.size[0] / sample_size[0])):
            for y in range(int(img.size[1] / sample_size[1])):
                left = x * sample_size[0]
                top = y * sample_size[1]
                yield img.crop((left, top, left + sample_size[0], top + sample_size[1]))


def sources_of_texture(kind: str) -> Generator[str, None, list]:
    return (
        path_from_root("textures", kind, img)
        for img in os.listdir(path_from_root("textures", kind))
    )


SAMPLE_SIZE = 128
for kind in os.listdir(path_from_root("textures")):
    if not os.path.isdir(path_from_root("textures", kind)) or kind[0] == ".":
        continue

    out_folder = path_from_root("samples", kind)
    if os.path.exists(out_folder):
        shutil.rmtree(out_folder)
    os.mkdir(out_folder)

    sample_gen = (
        sample
        for src in sources_of_texture(kind)
        for sample in extract_samples_from_file(src, SAMPLE_SIZE)
    )
    for i, sample in enumerate(sample_gen):
        sample.save(path_from_root("samples", kind, f"{i}.jpg"))
    print(f"{i} samples created from {kind} textures")

449 samples created from wood textures
